# Измерение качества моделей

## Метрики качества

Это практическое задание посвящено ознакомлению с инструментами оценки качества моделей машинного обучения, которые предоставляет библиотека `scikit-learn`. Метрики качества, представленные различными функциями, находятся в модуле `sklearn.metrics`. Мы начнем с загрузки набора данных из файла `data.csv` при помощи функции `read_csv` из библиотеки `pandas`. Этот набор данных содержит информацию о предсказаниях различных алгоритмов машинного обучения для решения задачи классификации. Колонка `prediction` - это результаты работы одного из этих алгоритмов. Целевая переменная содержится в столбце `target` (класс 0 или 1). Подсчитайте значение `true negative`, `false negative`, `true positive` и `false positive`. Запишите эти значения через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой. Далее, посчитайте для этих данных значение таких метрик как `precision`, `recall` и `f1 score` с точностью до двух знаков после запятой. Запишите результаты в строго заданном порядке через запятую в переменную `answer2`, которая так же будет являться строкой

### *РЕШЕНИЕ*

In [10]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import sklearn.metrics as mt

In [4]:
data = pd.read_csv('data.csv')
data.head()

,prediction,scores_1,scores_2,target
0,1,0.89,0.9,1
1,0,0.06,0.0,0
2,1,0.71,0.6,0
3,1,1.00,0.7,1
4,1,1.00,1.0,1


In [65]:
m=confusion_matrix(data.target,data.prediction)
tn = data[(data.prediction == 0) & (data.target==0)].target.count()
fn = data[(data.prediction == 0) & (data.target == 1)].target.count()
tp = data[(data.prediction == 1) & (data.target == 1)].target.count()
fp = data[(data.prediction == 1) & (data.target == 0)].target.count()
answer1 = f'{tn},{fn},{tp},{fp}'
answer1

'63,4,114,7'

In [67]:
from sklearn.metrics import precision_score, recall_score, f1_score
precission = mt.precision_score(data.target, data.prediction)
recall = mt.recall_score(data.target, data.prediction)
f1_score = mt.f1_score(data.target, data.prediction)
answer2 = f'{precission},{recall},{f1_score}'
answer2 

'0.9421487603305785,0.9661016949152542,0.9539748953974895'

В столбцах `scores_1` и `scores_2` содержаться оценки вероятности пренадлежности объектов к классу 1 для двух разных алгоритмов машинного обучения. Рассчитайте площадь под ROC-кривой для каждого алгоритма и сравните их. В качестве ответа `answer3` приведите большее из двух значений, округленное до трех знаков после запятой.

### *РЕШЕНИЕ*

In [21]:
from sklearn.metrics import roc_auc_score
sc1 = roc_auc_score(data.target, data.scores_1)
sc2 = roc_auc_score(data.target, data.scores_2)
answer3 = round(max(sc1,sc2),3)

0.993

## Метод скользящего контроля

Во второй части данного практического задания мы изучать различные методы оценки моделей машинного обучения. Загрузите набор данных `Breast Cancer Wisconsin (Diagnostic)`, используя функцию `load_breast_cancer` из модуля `sklearn.datasets`. Этот датасет позволяет решать задачу предсказания рака груди по различным характеристикам опухоли. В данном случае, целевая переменная принимает два значения, соответствующие доброкачественной и злокачественной опухоли. Проверьте, является ли данная выборка сбалансированной.

In [22]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)
counts = pd.value_counts(y)

print("Is this a balanced dataset? {}".format(counts[1] == counts[0]))

Is this a balanced dataset? False


In [31]:
counts

1    357
0    212
dtype: int64

Первый метод, который мы будем использовать, - это случайное разбиение датасета на тренировочную и тестовую выборку с помощью функции `train_test_split` из `sklearn.model_selection` с параметрами `random_state=3` и `test_size=0.33`. Если выборка является несбалансированной передайте целевую переменную в эту функцию в качестве аргумента `stratify`.

Обучите логистическую регрессию (класс `LogisticRegression` из модуля `sklearn.linear_model`) с параметром конструктора `random_state=42` и метод K ближайших соседей (класс `KNeighborsClassifier` из модуля `sklearn.neighbors`) на тренировочной выборке. Оцените качество на тестовой выборке для каждой из моделей. В качестве метрики качества используйте `recall`. Какая из моделей показывает лучший результат? Ответом на это задание `answer4` является этот результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [87]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y,stratify=y,random_state= 3,test_size=0.33)
lg = LogisticRegression(random_state=42)
model1 =lg.fit(X_train,Y_train)
kn = KNeighborsClassifier()
model2 = kn.fit(X_train,Y_train)
recall1 = recall_score(Y_test,lg.predict(X_test))
recall2 = recall_score(Y_test,kn.predict(X_test))
answer4 = max(recall1,recall2)
answer4

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9703389830508474

Далее мы проведем оценку каждой из этих моделей в соответствии с методом скользящего контроля с помощью функции `cross_val_score` из модуля `sklearn.model_selection`. В качестве параметра кросс-валидации `cv` в этой функции используйте экземпляр класса `StratifiedKFold` из `sklearn.model_selection` с тремя разбиениями. 

Функция `cross_val_score` возвращает количество оценок, соответствующие числу разбиений.
В качестве итогового результата используете среднее значение полученных оценок с помощью метрики `recall`. Какая модель работает лучше в это случае? Какие выводы можно из этого сделать? Ответом на это задание `answer5` является лучший итоговый результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [89]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np
sk = StratifiedKFold()
cr1 = cross_val_score(model1,X,y,cv= sk,scoring='recall').mean()
cr2 = cross_val_score(model2,X,y,cv= sk,scoring='recall').mean()

answer5 = max(cr1,cr2)
answer5

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:624: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/hvidsmen/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9663865546218489

 Строка с ответами

In [77]:
output = """TN,FN,TP,FP = {0}
Precision,Recall,F1 Score = {1}
Best ROC AUC Score {2:.3f}
Random Split {3:.3f}
Cross Val Score {4:.3f}"""
print(output.format(answer1, answer2, answer3, answer4, answer5))

TN,FN,TP,FP = 63,4,114,7
Precision,Recall,F1 Score = 0.9421487603305785,0.9661016949152542,0.9539748953974895
Best ROC AUC Score 0.993
Random Split 0.975
Cross Val Score 0.949
